In [1]:
import numpy as np
import pandas as pd
import math
from scipy.stats import norm
from scipy.optimize import fsolve
from numpy.linalg import inv
import scipy.integrate

In [2]:
def p1(X,mu1):
    return norm.pdf(X,mu1,1)
def p2(X,mu2):
    return norm.pdf(X,mu2,1)
def f(X,lamda,mu1,mu2):
    return (lamda*p1(X,mu1)+(1-lamda)*p2(X,mu2))

In [3]:
N=1000
alpha=0.05
z=norm.ppf(1-alpha/2)
pi=math.pi
F_inverse=[]
H_inverse=[]
lamda_MLE=[]
mu1_MLE=[]
mu2_MLE=[]
while len(F_inverse)<N:
    n=100
    X=[]
    U=np.random.uniform(0,1,n)
    for i in range(n):
        X.append((fsolve(lambda x: 0.5*norm.cdf(x)+0.5*norm.cdf(x-1)-U[i], 0)[0]))
    def ML(paramlist):
        lamda,mu1,mu2=paramlist[0],paramlist[1],paramlist[2]
        s1=0
        s2=0
        s3=0
        for i in range(n):
            s1=s1-(p1(X[i],mu1)-p2(X[i],mu2))/(f(X[i],lamda,mu1,mu2))
            s2=s2-lamda*p1(X[i],mu1)*(X[i]-mu1)/(f(X[i],lamda,mu1,mu2))
            s3=s3-(1-lamda)*p2(X[i],mu2)*(X[i]-mu2)/(f(X[i],lamda,mu1,mu2))
        return s1,s2,s3
    lamda_hat,mu1_hat,mu2_hat=fsolve(ML,[0.5,0,1])
    if lamda_hat>0.0001 and lamda_hat<0.9999:
        lamda_MLE.append(lamda_hat)
        mu1_MLE.append(mu1_hat)
        mu2_MLE.append(mu2_hat)
    else:
        continue
    h11=0
    h12=0
    h13=0
    h22=0
    h23=0
    h33=0
    for i in range(len(X)):
        h11=h11+(1/n)*(p1(X[i],mu1_hat)-p2(X[i],mu2_hat))**2/(f(X[i],lamda_hat,mu1_hat,mu2_hat))**2
        h12=h12-(1/n)*(p1(X[i],mu1_hat)*(X[i]-mu1_hat)*f(X[i],lamda_hat,mu1_hat,mu2_hat)-p1(X[i],mu1_hat)*(X[i]-mu1_hat)*lamda_hat*(p1(X[i],mu1_hat)-p2(X[i],mu2_hat)))/(f(X[i],lamda_hat,mu1_hat,mu2_hat))**2
        h13=h13-(1/n)*(-p2(X[i],mu2_hat)*(X[i]-mu2_hat)*f(X[i],lamda_hat,mu1_hat,mu2_hat)-p2(X[i],mu2_hat)*(X[i]-mu2_hat)*(1-lamda_hat)*(p1(X[i],mu1_hat)-p2(X[i],mu2_hat)))/(f(X[i],lamda_hat,mu1_hat,mu2_hat))**2
        h22=h22-(1/n)*(lamda_hat*p1(X[i],mu1_hat)*(-1+(X[i]-mu1_hat)**2)*f(X[i],lamda_hat,mu1_hat,mu2_hat)-(p1(X[i],mu1_hat)*(X[i]-mu1_hat)*lamda_hat)**2)/(f(X[i],lamda_hat,mu1_hat,mu2_hat))**2
        h33=h33-(1/n)*((1-lamda_hat)*p2(X[i],mu2_hat)*(-1+(X[i]-mu2_hat)**2)*f(X[i],lamda_hat,mu1_hat,mu2_hat)-(p2(X[i],mu2_hat)*(X[i]-mu2_hat)*(1-lamda_hat))**2)/(f(X[i],lamda_hat,mu1_hat,mu2_hat))**2
        h23=h23+(1/n)*(p1(X[i],mu1_hat)*(X[i]-mu1_hat)*lamda_hat*p2(X[i],mu2_hat)*(X[i]-mu2_hat)*(1-lamda_hat))/(f(X[i],lamda_hat,mu1_hat,mu2_hat))**2
    H=[[h11,h12,h13],[h12,h22,h23],[h13,h23,h33]]
    a11= lambda x:(p1(x,mu1_hat)-p2(x,mu2_hat))**2/f(x,lamda_hat,mu1_hat,mu2_hat)
    a22= lambda x:(lamda_hat*(x-mu1_hat)*p1(x,mu1_hat))**2/f(x,lamda_hat,mu1_hat,mu2_hat)
    a33= lambda x:((1-lamda_hat)*(x-mu2_hat)*p2(x,mu2_hat))**2/f(x,lamda_hat,mu1_hat,mu2_hat)
    a12= lambda x:(p1(x,mu1_hat)-p2(x,mu2_hat))*(lamda_hat*(x-mu1_hat)*p1(x,mu1_hat))/f(x,lamda_hat,mu1_hat,mu2_hat)
    a13= lambda x:(p1(x,mu1_hat)-p2(x,mu2_hat))*((1-lamda_hat)*(x-mu2_hat)*p2(x,mu2_hat))/f(x,lamda_hat,mu1_hat,mu2_hat)
    a23 = lambda x:(lamda_hat*(x-mu1_hat)*p1(x,mu1_hat))*((1-lamda_hat)*(x-mu2_hat)*p2(x,mu2_hat))/f(x,lamda_hat,mu1_hat,mu2_hat)
    f11 = scipy.integrate.quad(a11, -35, 35)[0]
    f22 = scipy.integrate.quad(a22, -35, 35)[0]
    f33 = scipy.integrate.quad(a33, -35, 35)[0]
    f12 = scipy.integrate.quad(a12, -35, 35)[0]
    f13 = scipy.integrate.quad(a13, -35, 35)[0]
    f23 = scipy.integrate.quad(a23, -35, 35)[0]
    F=[[f11,f12,f13],[f12,f22,f23],[f13,f23,f33]]
    if 10000>inv(H)[0][0]>0 and 10000>inv(H)[1][1]>0 and 10000>inv(H)[2][2]>0 and 10000>inv(F)[0][0]>0 and 10000>inv(F)[1][1]>0 and 10000>inv(F)[2][2]>0:
        F_inverse.append(inv(F))
        H_inverse.append(inv(H))
    print(len(F_inverse))

1
2


C:\Users\jsh98\anaconda3\lib\site-packages\scipy\optimize\minpack.py:162: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)


2
3
4
5


C:\Users\jsh98\anaconda3\lib\site-packages\scipy\optimize\minpack.py:162: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


5
6
6
6
7
8
8
9
10
11
11
11
11
12
13
14
15
16
17
17
18
19
20
20
21
22
23
24
25
26
27
28
29
30
31
31
31
32
33
34
35
36
37
38
39
40
41
41
42
43
44
44
44
45
46
47
48
49
50
51
51
52
53
54
55
56
57
58
59
60
60
61
62
63
64
64
65
66
67
68
69
70
70
70
71
71
72
72
73
74
74
74
75
76
76
76
77
78
79
80
81
82
83
83
84
85
86
87
88
89
90
91
92
93
94
95
95
95
96
97
98
99
99
100
100
101
101
101
102
103
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
121
121
121
121
122
123
124
125
126
127
128
129
129
129
130
131
132
133
134
135
135
136
137
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
158
159
160
161
161
161
162
163
164
165
166
166
166
167
168
169
170
171
172
173
173
173
174
175
176
177
178
179
180
181
182
183
183
184
185
186
187
188
189
190
190
190
191
192
192
193
194
194
195
196
197
198
198
199
200
201
202
203
204
205
206
207
207
208
209
210
211
212
213
214
215
215
216
217
218
219
220
221
222
223
224
225
226
227
228
228
229
230
231

C:\Users\jsh98\anaconda3\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\jsh98\anaconda3\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\jsh98\anaconda3\lib\site-packages\ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in double_scalars


870
871
872
873
874
875
875
876
877
877
878
878
878
878
878
878
879
879
880
881
881
882
883
884
885
886
887
888
889
890
891
892
892
892
893
894
895
896
897
898
899
900
901
902
903
904
905
906
907
907
907
908
909
910
911
912
913
914
915
916
917
918
918
919
919
920
921
922
922
922
923
924
924
925
926
927
928
929
930
930
931
932
933
934
935
935
936
937
938
939
939
940
940
941
941
942
942
943
944
945
946
946
946
947
948
948
949
950
951
952
953
953
954
954
955
955
956
957
957
957
958
959
959
960
961
962
963
964
965
966
967
968
968
968
969
970
971
972
973
973
974
975
975
976
977
978
978
978
978
979
980
981
981
982
983
984
985
986
987
987
988
989
990
991
992
993
994
995
996
997
998
999
1000


In [24]:
mu1_MLE

[0.10917152202822013,
 -0.25744253287128416,
 0.5782235862283449,
 0.382289089260745,
 -0.5877002492087838,
 -0.08473800703780261,
 -0.1843801682855476,
 -0.054126214487269515,
 0.38779634097614213,
 0.46125809376343385,
 0.34057660981736476,
 0.09660606041102962,
 0.3455616368762592,
 -0.42616326601136023,
 0.24014768813640738,
 -0.26546340595811635,
 -0.013273850904691336,
 0.4684151026197445,
 0.3923864026920001,
 0.06497887471620946,
 -0.21934326678644334,
 -0.050572061026308485,
 0.10501416119572123,
 -0.049479486197546906,
 -0.2396052814606039,
 0.19342308757627188,
 -0.986362387056363,
 -0.11571370708017857,
 -0.11680544519246638,
 0.21155381666364986,
 0.07788574606901755,
 0.14067350379762072,
 -0.8015146999984819,
 -0.6649143400615616,
 -0.19950326226687362,
 0.24720803738766398,
 0.21233389504087688,
 -0.22118276203231121,
 0.3130309558330763,
 -0.08739484664108299,
 0.025835172670037345,
 0.5734726732822353,
 0.36518253758722474,
 0.24748314374061356,
 -0.051947741322247054

In [5]:
mu2_MLE

[1.1887600886616636,
 1.100883530875247,
 0.8298732741843661,
 2.3470251676814167,
 1.0466264290526388,
 1.1032776489934246,
 0.48257736578259586,
 0.6778525946927547,
 0.45218395530171107,
 1.2742147884947415,
 1.3964060044559201,
 1.1429464951273196,
 1.9596342259998485,
 0.6327224335443756,
 1.3560090176005197,
 1.2360752243660944,
 0.7872230331655067,
 0.4684151026480787,
 1.1073873808762371,
 0.7611000884301464,
 0.7398006849274849,
 1.3196011792103952,
 1.2569825877218683,
 1.4859746034053447,
 1.0315373830230679,
 0.19342308757925203,
 0.5930298466442332,
 1.129491882468405,
 0.6296896019128838,
 1.2391737070973672,
 1.502800098505494,
 1.100144344310234,
 0.626379191598037,
 0.8578393533796831,
 1.0696091218445076,
 1.733985576703944,
 2.571381714815098,
 1.0390244977968304,
 2.281239196840219,
 0.855469601738786,
 0.6317749055557522,
 0.752387350689911,
 0.9496833392210418,
 2.1784924521010636,
 1.0484648537418584,
 0.9444554687152993,
 0.6552133464241755,
 1.201152206673853,


In [6]:
lamda_MLE

[0.6482770294683281,
 0.49570852835563906,
 0.5255004641446298,
 0.8865701571897834,
 0.3149425857432761,
 0.3254693392365495,
 0.36219914311260226,
 0.5873930473045141,
 0.1546079639119716,
 0.8570351875634845,
 0.8635463409773958,
 0.4704175305380051,
 0.9834053038616503,
 0.13774483622629527,
 0.889042761995524,
 0.556554754498245,
 0.21166970448109765,
 0.9929665588540395,
 0.6407657572688448,
 0.09984977732665119,
 0.2796336872215985,
 0.6773909872006829,
 0.5956475120796598,
 0.7373488397537329,
 0.3506214377788516,
 0.2809886517738447,
 0.13905428109916365,
 0.3924943428798375,
 0.2704888474955946,
 0.8144706647280575,
 0.8129596909555085,
 0.4658172250130615,
 0.17645585523536853,
 0.32761633875557883,
 0.3424353110182898,
 0.7893507520909328,
 0.869653749662562,
 0.4017340334287844,
 0.8863178343663676,
 0.529269661827097,
 0.5374513135201203,
 0.8841601690133207,
 0.6167157184917076,
 0.9169975661965755,
 0.2725623488190969,
 0.23555826702815225,
 0.19098846638143466,
 0.5286

In [16]:
H_inverse

[array([[14.979866  , 12.88192692, 21.53647661],
        [12.88192692, 13.81003327, 16.92938524],
        [21.53647661, 16.92938524, 37.74448487]]),
 array([[ 5.16110816,  6.89222053,  6.45336857],
        [ 6.89222053, 12.77569494,  7.98915324],
        [ 6.45336857,  7.98915324, 11.5121664 ]]),
 array([[ 1.12985051,  1.33481391,  7.39550628],
        [ 1.33481391,  2.93361679,  9.00732928],
        [ 7.39550628,  9.00732928, 66.51326031]]),
 array([[ 2.01455605,  4.43470305,  2.25259037],
        [ 4.43470305, 15.79358171,  4.77869733],
        [ 2.25259037,  4.77869733,  4.63676938]]),
 array([[ 35.81665483,  60.75637227,  33.37964578],
        [ 60.75637227, 109.43787285,  55.684421  ],
        [ 33.37964578,  55.684421  ,  33.40392273]]),
 array([[61.65424451, 39.43682652, 52.80395409],
        [39.43682652, 30.2047018 , 29.38605229],
        [52.80395409, 29.38605229, 54.26523351]]),
 array([[ 30.85554123,  20.59885128, 107.47760238],
        [ 20.59885128,  15.41125495,  69.1849

In [8]:
F_inverse

[array([[25.67049687, 22.05178787, 37.44920652],
        [22.05178787, 21.59015775, 30.73570861],
        [37.44920652, 30.73570861, 61.1403949 ]]),
 array([[ 8.18680644, 10.85897491, 10.70394782],
        [10.85897491, 17.86083389, 13.6751489 ],
        [10.70394782, 13.6751489 , 17.37202366]]),
 array([[ 0.96538443,  1.14261772,  6.04897326],
        [ 1.14261772,  2.71499315,  7.3870362 ],
        [ 6.04897326,  7.3870362 , 55.85399744]]),
 array([[ 3.14337579,  7.06541535,  3.73616951],
        [ 7.06541535, 21.689177  ,  8.34433902],
        [ 3.73616951,  8.34433902,  6.53687371]]),
 array([[15.42413234, 26.27784676, 14.09967257],
        [26.27784676, 51.44307116, 22.94239815],
        [14.09967257, 22.94239815, 15.24554062]]),
 array([[211.20690161, 133.44991864, 184.27711234],
        [133.44991864,  88.76130538, 112.80648223],
        [184.27711234, 112.80648223, 168.74460656]]),
 array([[ 23.81400991,  16.15576816,  81.11079322],
        [ 16.15576816,  12.62232373,  52.4557

In [9]:
v11=np.cov(lamda_MLE,mu1_MLE)[0][0]*100
v12=np.cov(lamda_MLE,mu1_MLE)[0][1]*100
v22=np.cov(lamda_MLE,mu1_MLE)[1][1]*100
v13=np.cov(lamda_MLE,mu2_MLE)[0][1]*100
v33=np.cov(lamda_MLE,mu2_MLE)[1][1]*100
v23=np.cov(mu1_MLE,mu2_MLE)[0][1]*100

In [10]:
V=[[v11,v12,v13],[v12,v22,v23],[v13,v23,v33]]

In [11]:
V

[[6.393350619497662, 8.189125665565152, 8.805603156694987],
 [8.189125665565152, 18.64558479448152, 8.874494870138466],
 [8.805603156694987, 8.874494870138466, 20.892116205795546]]

In [17]:
from numpy import linalg as LA
dF=[]
dH=[]
for i in range(len(F_inverse)):
    dF.append(LA.norm(F_inverse[i]-V))
    dH.append(LA.norm(H_inverse[i]-V))
mF=np.argsort(dF)[len(dF)//2]
mH=np.argsort(dH)[len(dH)//2]

In [18]:
F_inverse[mF]

array([[ 7.61254675, 20.01097431,  7.07889989],
       [20.01097431, 62.51924011, 17.81166318],
       [ 7.07889989, 17.81166318,  8.38612861]])

In [19]:
H_inverse[mH]

array([[14.93538103, 11.96607421, 23.18615522],
       [11.96607421, 12.0862691 , 16.6543464 ],
       [23.18615522, 16.6543464 , 44.96118175]])

In [12]:
gh11=[]
gf11=[]
gh22=[]
gf22=[]
gh33=[]
gf33=[]
for i in range(len(F_inverse)):
    gh11.append(2*norm.cdf(z*(H_inverse[i][0][0]/v11)**0.5)-1)
    gf11.append(2*norm.cdf(z*(F_inverse[i][0][0]/v11)**0.5)-1)
    gh22.append(2*norm.cdf(z*(H_inverse[i][1][1]/v22)**0.5)-1)
    gf22.append(2*norm.cdf(z*(F_inverse[i][1][1]/v22)**0.5)-1)
    gh33.append(2*norm.cdf(z*(H_inverse[i][2][2]/v33)**0.5)-1)
    gf33.append(2*norm.cdf(z*(F_inverse[i][2][2]/v33)**0.5)-1)

In [13]:
MSEh11=np.mean((gh11-np.asarray(0.95))**2)
MSEf11=np.mean((gf11-np.asarray(0.95))**2)
print(MSEh11)
print(MSEf11)
print(MSEh11/MSEf11)

0.027388858600287128
0.024759686482267527
1.1061876175169896


In [14]:
MSEh22=np.mean((gh22-np.asarray(0.95))**2)
MSEf22=np.mean((gf22-np.asarray(0.95))**2)
print(MSEh22)
print(MSEf22)
print(MSEh22/MSEf22)

0.022543976886183703
0.019121778407106913
1.178968629706789


In [15]:
MSEh33=np.mean((gh33-np.asarray(0.95))**2)
MSEf33=np.mean((gf33-np.asarray(0.95))**2)
print(MSEh33)
print(MSEf33)
print(MSEh33/MSEf33)

0.02243063416076068
0.018755445462294254
1.1959531542908424


In [20]:
chi_F=[]
chi_H=[]
chi_V=[]
for i in range(1000):
    theta=[[lamda_MLE[i]-0.5,mu1_MLE[i]-0.0,mu2_MLE[i]-1]]
    chi_V.append(np.dot(theta,np.dot(n*inv(V),np.transpose(theta))))
    chi_H.append(np.dot(theta,np.dot(n*inv(H_inverse[i]),np.transpose(theta))))
    chi_F.append(np.dot(theta,np.dot(n*inv(F_inverse[i]),np.transpose(theta))))

In [21]:
np.mean(chi_V)

3.2251700842033677

In [22]:
np.mean(chi_F)

9.859271211976157

In [23]:
np.mean(chi_H)

9.997614259177976